### <span style="color:red">IMPORTANT: Only modify cells which have the following comment:</span>
```python
# Modify this cell
```
##### <span style="color:red">Do not add any new cells when you submit the homework</span>

# Setting Up Notebook

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
sc = SparkContext(master="local[4]")

In [3]:
import numpy as np
import math
from numpy import linalg as LA

# Exercise:
The function **computeCov** computes the covariance matrix using RDDs. The code allows undefined entries and calculates the covariance without bias. 

Your homework is to complete the missing parts in **computeCov** (Marked with `...`) so that it calculates the covariance correctly.

    Note: The functions and libraries in the cell below will be useful to you

In [5]:
def outerProduct(X):
    """Computer outer product and indicate which locations in matrix are undefined"""
    O=np.outer(X,X)
    N=1-np.isnan(O)
    return (O,N)

def sumWithNan(M1,M2):
    """Add two pairs of (matrix,count)"""
    (X1,N1)=M1
    (X2,N2)=M2
    N=N1+N2
    #compute X1 + X2
    X=np.nansum(np.dstack((X1,X2)),axis=2)
    return (X,N)

In [7]:
# Modify this cell

def computeCov(RDDin):
    # input: RDDin is an RDD of np arrays, all of the same length
    
    # we insert 1 at the beginning of each vector so the calculation also yields the mean vector
    RDD=RDDin.map(lambda v:np.array(np.insert(v,0,1),dtype=np.float64)) 
    
    # separating map and reduce does not matter, since Spark uses lazy execution.
    OuterRDD = RDD.map(lambda v: outerProduct(v))    #<-- do mapping here
    (S,N) = OuterRDD.reduce(lambda x, y: sumWithNan(x, y))  #<-- do reducing here
    
    # Unpack result and compute the covariance matrix
#    print 'RDD=',RDD.collect()
#    print 'shape of S=',S.shape,'shape of N=',N.shape
#    print 'S=',S
#    print 'N=',N
    E=S[0,1:]
    NE=np.float64(N[0,1:])
    # mean vector
    Mean=E / NE
    # outer product of mean vector
    (Sub, SN) = outerProduct(Mean)

    O = S[1:,1:]
    NO = np.float64(N[1:,1:])
    Sum = O / NO
    
    Cov = Sum - Sub 
    # This is the covariance matrix
    
    # Output also the diagnal which is the variance for each day
    Var=np.array([Cov[i,i] for i in range(Cov.shape[0])])
    print "E"
    print E
    return {'E':E,'NE':NE,'O':O,'NO':NO,'Cov':Cov,'Mean':Mean,'Var':Var}


In [8]:
import Tester.SmallPCA as pca
pca.exercise(computeCov, sc)

Checking data_list of length 3 with length 10 vectors each having 2 np.NaN values
E
[-1.20959365 -1.33021144  0.66385386  1.14962439  0.34924947 -0.44321848
  0.50380628  0.64500887 -0.20572369 -1.55610481]

Checking data_list of length 100 with length 10 vectors each having 4 np.NaN values
E
[ 4.59824787  0.13934832 -0.15725499  4.10758454  0.8717172  -0.61126751
 -0.13013131  5.26389931  0.38472213  0.44514038]

Great Job!
